In [1]:
import os
import utils
import argparse
import numpy as np
import pandas as pd
import compress_pickle as cpickle
     

from datetime import date
from datetime import datetime

import sqlalchemy as sql

import warnings
warnings.filterwarnings('ignore')


In [4]:
def prepare_dataframe(dataframe, has_user=True, has_fcreate=False, has_fmod=False):
    if has_user:
        dataframe['UsuarioModificacion'] = "@usuariomodificacion"
    if has_fcreate:
        dataframe['FechaCreacion'] = "@fechacreacion"
    if has_fmod:
        dataframe['FechaModificacion'] = "@fechamodificacion"
    return dataframe        

def prepare_sqlinput_rows(dataframe):
    
    rows = []
    for index, row in dataframe.iterrows():
        rows.append(row)

    dfvalues = ""

    for row in rows:

        temp_dict = row.to_dict()
        
        row_values = '('

        for k in temp_dict.keys():

            value = temp_dict[k]
            
            if type(value) == str:

                if value[0] == '@':
                    row_values = row_values + value + ", "
                else:
                    value = value.replace("'", "")
                    row_values = row_values + "'" + value + "', "
            
            elif type(value) == int:
                row_values = row_values + str(value) + ", "
                            
        row_values = "    " + row_values[:-2] + '),\n'
        
        dfvalues = dfvalues + row_values

    dfvalues = dfvalues[:-2] + ';'

    return dfvalues

def GET_SQL_INPUT(dataframe, table_name, use_database=True, database_name='Dbo', use_schema=True, schema_name='Dbo'):
    
    n = dataframe.shape[0]

    insert_cols = "("
    for c in dataframe.columns.tolist():
        insert_cols += c + ", "
    insert_cols = insert_cols[:-2] + ")"

    insert_block = "INSERT INTO "

    if use_database:
        insert_block += "[" + database_name + "]."
    if use_schema:
        insert_block += "[" + schema_name + "]."

    insert_block += "[" + table_name + "] " + insert_cols + "\nVALUES\n"


    if n <= 1000:

        dfvalues = prepare_sqlinput_rows(dataframe)
        output = insert_block + dfvalues

    else:

        s = int(np.floor(n / 1000))
        
        cdfs = []
        for i in range(s+1):
            cdfs.append(dataframe.iloc[i*1000:(i+1)*1000,:])

        output = ""
        for df in cdfs:
            dfvalues = prepare_sqlinput_rows(df)
            output = output + insert_block + dfvalues + '\n\n'

        output = output[:-2]
    
    return output

In [5]:
data = {'Codigo': [1, 2, 3, 4, 5, 6],
        'Glosa': ['ABC', 'ABC', 'ABC', 'ABC', 'FSAL', 'DPR']}

df = pd.DataFrame(data)
sqls = GET_SQL_INPUT(df,table_name='TemporaryUploadTest', database_name="DesarrolloBI", schema_name="Prestaciones")
sql = 'USE DesarrolloBI ' + 'GO' +' CREATE TEMPORARY TABLE TemporaryUploadTest (Codigo int, Glosa varchar(5)) '+ sqls + '\n\n'
today = date.today()
today = today.strftime('%Y%m%d')

now = datetime.now()
now = now.strftime('%H%M')

filename = 'output/create_upload_script/'+ 'UploadDesarrolloClasificacionPrestacion' + '_' + today + '_' + now + '.sql'

with open(filename, 'w') as f:
    f.write(sql)        
    
